In [53]:
# Sam Brown
# sam_brown@mines.edu
# July 8
# Goal: Investigate different ways to calculate the energy of a slip event and look for trends 

import pandas as pd
import numpy as np
from Stations import Station
import my_lib.funcs
import matplotlib.pyplot as plt

from netCDF4 import Dataset
import xarray as xr

In this notebook I will be calculating the energy of a slip using a simple energy formula(s). 
$K = \frac{1}{2}m v^2$
There are existing formulas in the realm of seismology for calculating stress drop. For a circular fault in a whole space, we can estimate the stress with the equation
$$
\Delta \sigma = \frac{7 \pi \mu \bar{D}}{16r} = \frac{7 M_0}{16r^3}
$$
Where r is the fault radius, $\mu$ is the shear modulus, and $\bar{D}$ is the average displacement. Overall, these equations are important (when simplified) as they model the idea that the shear stress change on the fault will be proportional to the ratio of the displacement to the size of the fault.

For our Calculations we will start by investigating the energy of the events that occur in the grounding zone (GZ) area. First we will need to calculate of the volume of the ice in that area. We will use a circle with a radius of 15 km centered at gz07. 

In [6]:
# Load in Antarctica ice depth data
filepath = "/Users/sambrown04/Documents/SURF/NSIDC-0756_3-20250708_204745/BedMachineAntarctica-v3.nc"
ds = xr.open_dataset(filepath)

In [7]:
print(ds)

<xarray.Dataset> Size: 4GB
Dimensions:    (x: 13333, y: 13333)
Coordinates:
  * x          (x) int32 53kB -3333000 -3332500 -3332000 ... 3332500 3333000
  * y          (y) int32 53kB 3333000 3332500 3332000 ... -3332500 -3333000
Data variables:
    mapping    |S1 1B ...
    mask       (y, x) int8 178MB ...
    firn       (y, x) float32 711MB ...
    surface    (y, x) float32 711MB ...
    thickness  (y, x) float32 711MB ...
    bed        (y, x) float32 711MB ...
    errbed     (y, x) float32 711MB ...
    source     (y, x) int8 178MB ...
    dataid     (y, x) int8 178MB ...
    geoid      (y, x) int16 356MB ...
Attributes: (12/17)
    Conventions:                 CF-1.7
    Title:                       BedMachine Antarctica
    Author:                      Mathieu Morlighem
    version:                     03-Jun-2022 (v3.4)
    nx:                          13333.0
    ny:                          13333.0
    ...                          ...
    ymax:                        3333000
  

In [8]:
gz07x = -168846.09042414097 
gz07y = -602168.8273598101

gz05x = -155766.98797266872 
gz05y = -605038.2386205866

gz01x = -178533.70585008597 
gz01y = -610921.0441554557

gz18x = -167907.0049366487 
gz18y = -588200.1441347648

In [9]:
coords = {
    'gz07': (-168846.09042414097, -602168.8273598101),
    'gz05': (-155766.98797266872, -605038.2386205866),
    'gz01': (-178533.70585008597, -610921.0441554557),
    'gz18': (-167907.0049366487, -588200.1441347648)
}

for label, (x_val, y_val) in coords.items():
    thickness = ds['thickness'].sel(x=x_val, y=y_val, method='nearest')
    print(f"{label}: {thickness.values:.2f} meters")

gz07: 730.12 meters
gz05: 738.91 meters
gz01: 759.08 meters
gz18: 750.39 meters


In [10]:
# Take an average of these points around our desired circle 
avg_depth = 744.63
volume = np.pi * 15000 ** 2 * avg_depth # m^3
density = 910 # kg/m^3
mass = volume * density
print(mass)

478976617182315.75


We have the mass now we need to retreieve velocity data at gz stations to calculate figures for kinetic energy. To do this, for each gz event we need to detect the slip and then calculate the velocity of each station after the event initiates, and do an average of these values.

In [12]:
# Start by loading in data on gz stations from 2011 to 2013
df_2011 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2011_2011Events2stas")
df_2012 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2012_2012Events2stas")
df_2013 = my_lib.funcs.load_evt("/Users/sambrown04/Documents/SURF/Events/2013_2013Events2stas")

df = df_2011 + df_2012 + df_2013

In [13]:
df = my_lib.funcs.preprocess_events(df)

In [15]:
# Filter so it is only gz stations
gz_dat = []

for i, event in enumerate(df):

    filt_df = pd.DataFrame()
    
    for col in event.columns:
        colname = col[0:2]
        if colname == 'gz' or colname == 'ti' or colname == 'ti':
            column = event[col].copy()
            filt_df[col] = column
            
    gz_dat.append(filt_df)

In [16]:
# Loop through each station and calculate the onset of the station

f_events = []
for i, event in enumerate(gz_dat):
    cols = [col for col in event.columns if (col != 'time_sec' and col != 'time_dt')]
  
    event_onsets = []e 
    for col in cols:
        
        # Compute first and second derivatives of the station's signal
        grad = my_lib.funcs.derivative(event[col].values)
        grad2 = my_lib.funcs.derivative(grad)

        # Identify the peak in the second derivative (maximum acceleration)
        max_idx = np.argmax(np.abs(grad2))
        max_time = event['time_sec'].iloc[max_idx]
        event_onsets.append(max_time)
        # print(max_time)


    if not event_onsets:
        print(f"Skipping event: {i} no valid signals for onset detection.")
        continue  # skip this event
    
    release_time = max(event_onsets)
       
    # Splice DataFrame starting from release_time
    spliced_event = event[event['time_sec'] >= release_time].reset_index(drop=True)
    f_events.append(spliced_event)

Skipping event: 17 no valid signals for onset detection.
Skipping event: 29 no valid signals for onset detection.
Skipping event: 46 no valid signals for onset detection.
Skipping event: 52 no valid signals for onset detection.
Skipping event: 141 no valid signals for onset detection.
Skipping event: 269 no valid signals for onset detection.
Skipping event: 360 no valid signals for onset detection.
Skipping event: 395 no valid signals for onset detection.
Skipping event: 417 no valid signals for onset detection.
Skipping event: 422 no valid signals for onset detection.
Skipping event: 423 no valid signals for onset detection.
Skipping event: 449 no valid signals for onset detection.
Skipping event: 459 no valid signals for onset detection.
Skipping event: 463 no valid signals for onset detection.
Skipping event: 478 no valid signals for onset detection.


In [66]:
derivs_events = []

for i, event in enumerate(f_events):
    cols_avg = [col for col in event.columns if col not in ['time_sec', 'time_dt']]
    f_events[i]['avg_disp'] = event[cols_avg].mean(axis=1)

    if f_events[i].shape[0] < 50:
        print(f_events[i].shape[0])
        continue

    # Calculate derivative
    deriv = my_lib.funcs.derivative(f_events[i]['avg_disp'])
    
    # Get corresponding date (first entry in time_dt column)
    date = event['time_dt'].iloc[0]

    # Append (date, derivative) tuple
    derivs_events.append((date, deriv))


2
48
7
7
9
3
11
4
9
2
5
5
1
2


In [47]:
def event_energy(velocity_arr, mass, dt=15):
    kinetic_energy = 0.5 * mass * velocity_arr**2  # joules at each step
    total_energy = np.sum(kinetic_energy) * dt       # Approx integration
    return total_energy  # in joules

In [69]:
energies = pd.DataFrame(columns = ['date', 'evt_energy'])

dt = 15
mass = mass

for event in derivs_events:
    energies.loc[len(energies)] = {
        "date": event[0],
        "evt_energy": event_energy(event[1], mass, dt)
    }

In [73]:
energies = energies.sort_values('date')

In [83]:
f_events = []
derivs_events = []
energies = pd.DataFrame(columns=['date', 'evt_energy'])

dt = 15  # Time step in seconds
mass = mass  # Assumed to be defined earlier

for i, event in enumerate(gz_dat):
    cols = [col for col in event.columns if col not in ['time_sec', 'time_dt']]
    event_onsets = []

    for col in cols:
        # Compute first and second derivatives
        grad = my_lib.funcs.derivative(event[col].values)
        grad2 = my_lib.funcs.derivative(grad)

        # Identify max of second derivative (absolute value)
        max_idx = np.argmax(np.abs(grad2))
        max_time = event['time_sec'].iloc[max_idx]
        event_onsets.append(max_time)

    # Skip if no valid onset found
    if not event_onsets:
        print(f"Skipping event {i}: no valid onsets found.")
        continue

    # Store date from original (unspliced) event
    date = event['time_dt'].iloc[0]

    # Determine the release time as the max of onsets
    release_time = max(event_onsets)

    # Splice the event starting from release_time
    spliced_event = event[event['time_sec'] >= release_time].reset_index(drop=True)

    # Skip if spliced event too short
    if spliced_event.shape[0] < 50:
        print(f"Skipping short event {i}: only {spliced_event.shape[0]} samples.")
        continue

    # Compute average displacement
    cols_avg = [col for col in spliced_event.columns if col not in ['time_sec', 'time_dt']]
    spliced_event['avg_disp'] = spliced_event[cols_avg].mean(axis=1)

    # Compute derivative of average displacement
    deriv = my_lib.funcs.derivative(spliced_event['avg_disp'])

    # Store spliced event
    f_events.append(spliced_event)

    # Store (date, derivative) pair
    derivs_events.append((date, deriv))

    # Compute and store energy
    evt_E = event_energy(deriv, mass, dt)
    energies.loc[len(energies)] = {
        "date": date,
        "evt_energy": evt_E
    }

Skipping event 17: no valid onsets found.
Skipping event 29: no valid onsets found.
Skipping event 46: no valid onsets found.
Skipping event 52: no valid onsets found.
Skipping short event 132: only 2 samples.
Skipping short event 140: only 48 samples.
Skipping event 141: no valid onsets found.
Skipping event 269: no valid onsets found.
Skipping event 360: no valid onsets found.
Skipping event 395: no valid onsets found.
Skipping event 417: no valid onsets found.
Skipping event 422: no valid onsets found.
Skipping event 423: no valid onsets found.
Skipping event 449: no valid onsets found.
Skipping event 459: no valid onsets found.
Skipping event 463: no valid onsets found.
Skipping event 478: no valid onsets found.
Skipping short event 640: only 7 samples.
Skipping short event 726: only 7 samples.
Skipping short event 884: only 9 samples.
Skipping short event 910: only 3 samples.
Skipping short event 943: only 11 samples.
Skipping short event 1236: only 4 samples.
Skipping short event

In [91]:
energies = energies.sort_values('date')

In [100]:
energies.to_csv('Energies_gz', index = False)